<a href="https://colab.research.google.com/github/tobiasgobel/VQE_project/blob/master/VQE_grid_opt_clifford_gates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from sympy.series import series
from scipy.optimize import minimize, NonlinearConstraint
import time
import itertools
from sympy import symbols, Matrix, SparseMatrix, cos, sin, expand, lambdify, O
from sympy.utilities.iterables import multiset_permutations
from scipy.linalg import expm, sinm, cosm
from functools import *
from operator import *
import scipy
import sympy
import numpy as np
import random
import math
from numba import jit
import matplotlib.pyplot as plt

In [10]:
from functools import wraps
from time import time

def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print('func:%r  took: %2.4f sec' %(f.__name__, te-ts))
        return result
    return wrap

In [11]:

def pauli_on_pauli(p1,p2):
    
    if p1 == 'X' and p2 == 'Y':
        return 1j, 'Z'
    elif p1 == 'X' and p2 == 'X':
        return 1, 'I'
    elif p1 == 'Y' and p2 == 'Y':
        return 1, 'I'
    elif p1 == 'Z' and p2 == 'Z':
        return 1, 'I'
    elif p1 == 'Z' and p2 == 'X':
        return 1j, 'Y'
    elif p1 == 'Z' and p2 == 'Y':
        return -1j, 'X'
    elif p1 == 'I':
        return 1, p2
    elif p2 == 'I':
        return 1, p1
    else:
        a, p = pauli_on_pauli(p2,p1)
        return -1*a, p

def single_pauli_action(pauli, spin):
    
    if pauli=='X':
        return((spin+1)%2, 1)
    elif pauli=='Y':
        return((spin+1)%2, 1j*(-1)**spin)
    elif pauli=='Z':
        return(spin, (-1)**spin)
    elif pauli=='I':
        return(spin, 1)
    else:
        print('wrong pauli!')
        return(None)

def findCombinationsUtil(li, arr, index, num, reducedNum):
    z = []
    if (reducedNum < 0): 
        return; 
    if (reducedNum == 0): 
  
        for i in range(index): 
            z = z + [arr[i]]
        li.append(z) 
        return;

    prev = 1 if (index == 0) else arr[index - 1]; 
  
    for k in range(prev, num + 1): 
          

        arr[index] = k; 
  
        findCombinationsUtil(li,arr, index + 1, num,  
                                 reducedNum - k); 
    return li

def k_all(N, generators, order): 
      
    # array to store the combinations 
    # It can contain max n elements
    out = []
    k_length = len(generators)
    for k in range(1, order+1):
        arr = [0] * k;
        output = []
        a =  findCombinationsUtil([], arr, 0, k, k);
        for i in a:
            if len(i)<= k_length:
                i = i.extend((k_length-len(i))*[0])
        for j in a:
            if len(j) == k_length:
#                 if k_vector(N, interactions,j).state()[1] != N*[0]:
                output = output + list(multiset_permutations(j))
        out =  out+ output
    return [tuple(p) for p in [[0]*k_length] + out]

def power_product(x,y):
    out = 1
    for i in range(len(x)):
         out*= x[i]**y[i]
    return out



In [12]:
@timing
@jit(nopython=True)
def Energy_eigen(H):
  result = np.linalg.eig(H)
  index = np.argmin(result[0])
  return result[0][index],result[1][index]

def Energy_matrix(thetas,N,H,ansatz, K):

  #build psi
  a = np.eye(2**N)
  zero_state = np.zeros(2**N)
  zero_state[0]=1
  for i in range(len(ansatz)-1,-1,-1):
    T = ansatz[i]
    exp = expm(1j*(np.pi/4*K[i]+thetas[i])*T)
    a = a @ exp
  

  psi = a @ zero_state
  
  #build Hamiltonian
  Energy = (np.transpose((np.conj(psi)) @ (H @ (psi))))

  return np.real(Energy)

def psi(thetas, ansatz,K):
    #build psi
  a = np.eye(2**N)
  zero_state = np.zeros(2**N)
  zero_state[0]=1
  for i in range(len(ansatz)-1,-1,-1):
    T = ansatz[i]
    exp = expm(1j*(np.pi/4*K[i]+thetas[i])*T)
    a = a @ exp
  

  psi = a @ zero_state
  
  return psi

  

In [13]:
Pauli = {"X":np.array([[0,1],[1,0]]), "Z": np.array([[1,0],[0,-1]]),"Y":np.array([[0,-1j],[1j,0]]), "I":np.eye(2)}

class pauli:
  def __init__(self,string, N, factor = 1):
    self.string = string
    self.factor = factor
    self.N = N
    self.starting_state = np.array([0]*self.N)


  def __str__(self):
    return self.string+".   factor: "+str(self.factor)
    
  #define multiplying by a constant (on left hand side)
  def __rmul__(self, c):
    return pauli(self.string,self.N, c*self.factor)

  #define the power of a pauli string
  def __pow__(self, c): 
    C = pauli("I0",self.N)
    for i in range(abs(c)):
      C = C*self
    return C

  #define multiplying two pauli strings
  def __mul__(self, x):
    pos1, pauli1 = self.split()
    pos2, pauli2 = x.split()
    factor = self.factor*x.factor
    string = ""
    counter1 =0
    counter2 =0

    for j in range(self.N):
      end1 = counter1 == len(pos1)
      end2 = counter2 == len(pos2)

      if not end1 and not end2:
        if int(pos1[counter1]) == j and int(pos2[counter2]) == j:
          a, p= pauli_on_pauli(pauli1[counter1],pauli2[counter2])
          factor *= a
          string+= p+str(j)
          counter1+=1
          counter2+=1
        elif int(pos1[counter1]) == j:
          string+=pauli1[counter1]+str(j)
          counter1+=1
        elif int(pos2[counter2]) == j:
          string+=pauli2[counter2]+str(j)
          counter2+=1
      elif not end1:
        if int(pos1[counter1]) == j:
          string+=pauli1[counter1]+str(j)
          counter1+=1
      elif not end2:
          if int(pos2[counter2]) == j:
            string+=pauli2[counter2]+str(j)
            counter2+=1
      else:
        pass
      
    return pauli(string, self.N, factor)

  #calculate resulting state of paulistring when acted upon initial_state  
  def state(self, initial_state = 0):
    pos, pauli = self.split()
    init_state = self.starting_state + initial_state
    a = self.factor
    for j in range(len(pos)):
      Pauli = pauli[j]
      spin = init_state[int(pos[j])]
      new_spin, factor = single_pauli_action(Pauli,spin)
      init_state[int(pos[j])] = new_spin
      a *= factor
    return a, tuple(init_state)

    
#creating lists of operators and corresponding positions
  def split(self):
    pauli_lst = []
    pos_lst = []
    prev_int = False
    for k in self.string:
        if k.isdigit():
            if not prev_int:
                pos_lst.append(k)
            else:
                pos_lst[-1] += k
            prev_int = True
        else:
            pauli_lst.append(k)
            prev_int = False
    return pos_lst, pauli_lst
  
  def matrix_repr(self):
    positions, paulis = self.split()
    Kron = 1
    counter = 0
    for j in range(self.N):
      if counter == len(positions):
        for _ in range(self.N-j):
          Kron = np.kron(Kron, np.eye(2))
        break

      elif j == int(positions[counter]):
        Kron = np.kron(Kron, Pauli[paulis[counter]])
        counter+=1
      else:
          Kron = np.kron(Kron, np.eye(2))

    return Kron*self.factor
  
print(pauli("X0X1Y2",3,7).matrix_repr())
  

[[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-7.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+7.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-7.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+7.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-7.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+7.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.-7.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+7.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]]


In [14]:
#gives result of transformation exp(-i*T1)*T2*exp(i*T2)
def Clifford_map(T1, T2, reversed_arguments = True):
  global ansatz
  T1T2 = T1*T2
  T2T1 = T2*T1
  if T1T2.factor == T2T1.factor:
    if reversed_arguments:
      return T1
    else:
      return T2
  elif T1T2.factor == -T2T1.factor:
    if reversed_arguments:
      return -1j*T2T1
    else:
      return -1j*T1T2
  else:
    return "something wrong here"


#returns list of pauli objects that are the result 
#of pulling all clifford gates to the left
def pull_cliffords_through(ansatz, K, N):
  T_K = [ansatz[0]]
  
  for j in range(1, len(ansatz)):
    T = ansatz[j]
    for i in range(j-1,-1,-1):
      for _ in range(abs(K[i])):
        T = Clifford_map(T,ansatz[i])
    T_K += [T] 
  return T_K





In [15]:

def s_dict(N, ansatz, K, order):
  start = time()
  s_dict = {} #keys: possible bitstrings, values dictionary with orders
  T_K = pull_cliffords_through(ansatz, K, N)
  K_all = list(map(tuple, itertools.product([0, 1], repeat=len(ansatz))))

  for i in K_all: #loop through all
    if sum(i) > order:
      continue
    
    #calculate state that is produced by T_i
    pauli_string = power_product(T_K[::-1], i[::-1])
    factor, state = pauli_string.state()
    #calculate magnitude of term
    term = (1j)**sum(i)*factor
    #check whether binary string is in dictionary, otherwise add
    if state not in s_dict:
      s_dict[state] = ([list(i)],[term])
    else:
      current = s_dict[state]
      current[0].append(list(i))
      current[1].append(term)

  #make np.array
  for st in s_dict:
    lst = s_dict[st]
    s_dict[st] = (np.array(lst[0]),np.array(lst[1]))
  
  return s_dict


def G_k(N, H, ansatz, K):
  g_k = []

  #Initialize list of Clifford gates with respective power of K.
  G_K = []
  for i in range(len(K)):
    G_K += [np.sign(K[i])*ansatz[i]]*abs(K[i])
  for P in H:
    # G_K = [ansatz[i]**K[i] for i in range(len(K))]
    #Apply nested Clifford Map to obtain G^-K P_a G^K
    paulistring = reduce(Clifford_map, [P]+G_K[::-1])
    g_k += [paulistring]
  return g_k

@jit(nopython=True)
def dict_multiplication(k,values,thetas):
  sum = 0
  for i in range(k.shape[0]):
    product = 1
    for j in range(k.shape[1]):
      product*=(np.cos(thetas[j]))**(-k[i,j]+1)*(np.sin(thetas[j]))**k[i,j]
    sum += product*values[i]
  return sum

def Normalize(s_d, thetas, order):
  sum = 0
  for s in s_d:
    k, values = s_d[s]
    k1, values1 = s_d[s]
    factor = dict_multiplication(k,values,thetas)
    factor1 = dict_multiplication(k1,values1,thetas)
    sum += np.conj(factor1)*factor

  return sum



In [16]:
 
def energy(thetas,ansatz, s_dict,G_K, order):
  E = 0
  s_dict1 = s_dict
  for paulistring in G_K: #loop through terms in Hamiltonian
    E_a = 0
    #loop over basis states
    for s in s_dict1:
      E_a_s = 0
    
      #Calculate G^-K P_a G^K |s>
      a, state = paulistring.state(s)
      #Define contributions of |s> and |s'>
      psi_s1 = s_dict1[s]

      #Check if the state created by hamiltonian, exists in wavefunction
      try:
        psi_s2 = s_dict1[state]
      except:
        break

      A = dict_multiplication(psi_s1[0],psi_s1[1],thetas)
      B = dict_multiplication(psi_s2[0],psi_s2[1],thetas)
      E_a_s = A*np.conj(B)

      E_a_s *= a
      E_a += E_a_s
    E += E_a
  
  norm = Normalize(s_dict1, thetas, order)
  # print(f"E:{E}, Norm{norm}, E_final:{E/norm}")
  return np.real(E/norm)

In [17]:
def angle_compare(theta_opt, theta_appr, K):
  theta_appr = theta_appr + np.array(K)*np.pi/4
  theta_appr = theta_appr % (2*np.pi)
  theta_opt = theta_opt % (2*np.pi)

  distance = np.linalg.norm(theta_opt-theta_appr)
  
  return distance

def wavefunction_compare(theta_opt, theta_appr, K, ansatz):
  wave_1 = psi(theta_opt, ansatz, [0]*len(ansatz))
  wave_2 = psi(theta_appr, ansatz, K)
  distance = np.linalg.norm(wave_1-wave_2)
  return distance


K-cell finding
```
set number of iterations: iterations = iters
set initial K-cell: K_init = [0]*len(ansatz)
set order
set epsilon

for i in range(iters):

  calculate s_dict
  calculate G_K

  result = scipy.minimize(Energy, order, args = (s_dict, G_K)

  if biggest_angle > 1/8* pi:
      update K_init at index of biggest angle
  elif biggest_angle - 1/8* pi < epsilon:
      update K_init at index of biggest_angle
  else:  (all_angles < 1/8*pi)
    --> Valid K_cell
    return result

  if K_init in K_path:
      if there exists next_biggest angle > 1/8 * pi:
          update K_init at next_biggest angle
      elif previous K_init has next biggest angle > 1/8*pi:
          K_init = prev_K_init updated at next_biggest angle
      elif periodicity == 2:
          take average angle between two K-cells
      elif periodicity > 2:
          Output angles with lowest |theta|

(if run for all iterations without finding K-cell, take K_cell with lowest |theta|)
```







K-Cell finding ---- Minimizing magic gates

```
set number of K-cells one wants to consider: N_K
set K #initial K-cell
set order

K_tree = {(0):{"K":K, "seen":False}}
N_magic_prev = len(angles) #max magic gates
curr_tree = (0)

for _ in range(N_K):
    

    calculate s_dict
    calculate G_K

    result = scipy.minimize(energy, order, args = (s_dict, G_K))#bounded by |pi/8|

    N_magic = number of angles epsilon close to pi/8

    K_tree[curr_tree]["seen"] = True
    K_tree[curr_tree]["N_magic"]=N_magic

    

    # if Number of magic gates increases
    if N_magic > N_magic_prev:
        curr_tree, K = find_new_branch()
        break


    #include new branches 
    for i in range(N_magic):
        K_new_i = K but +/- 1 at i^th pi/8 angle
        K_tree[curr_tree+(i,)]={"K":K_new_i, "seen":False}

    #all angles within K-cell
    if N_magic == 0:
        break

    #choose new branch
    elif N_magic <= N_magic_prev:
        updated = False
        for i in range(N_magic):
            if K_i not in K_path:
                curr_tree += (i,)
                K = K_tree[curr_tree]["K"]
                updated = True
                break
        if not updated: #Only repeated gates
            curr_tree, K = find_new_branch()

        
def find_new_branch(K_tree, curr_tree, K_path):
    tree_up = curr_tree[:-1]

    if tree_up == (0):
        return "whole tree has been searched"

    N_magic = K_tree[tree_up]["N_magic"]

    updated = False

    for i in range(N_magic):
        tree_i = tree_up+=(i,)
        if not K_tree[tree_i]["seen"] and K_tree[tree_i]["K"] not in K_path:
            return tree_i, K_tree[tree_i]["K"]
    if not updated:
        find_new_branch(K_tree, tree_up, K_path)

```


In [18]:
def find_new_branch(K_tree, curr_tree, K_path):
    tree_up = curr_tree[:-1]

    if tree_up == (0):
        print("whole tree has been explored")
        return "whole tree has been searched"
    N_magic = K_tree[tree_up]["N_magic"]
    
    updated = False
    for i in range(N_magic):
        tree_i = tree_up+(i,)
        K_i = K_tree[tree_i]["K"]
        if not K_tree[tree_i]['seen'] and K_i not in K_path:
            updated = True
            return tree_i
    if not updated:
        return find_new_branch(K_tree, tree_up, K_path)

In [46]:
#initialize

N = 2
Z_h = -1
X_h = -.1
ansatz = [pauli("X0X1",N),pauli("Z0",N),pauli("Z1",N)]
H = [pauli("Z0",N,Z_h),pauli("Z0",N,Z_h)]
H+= [pauli("X0X1",N,X_h)]



N_K = 10
K = [0]*len(ansatz)
theta_init = [1]*len(ansatz)
order = len(ansatz)*2
K_tree = {(0,0):{"K":K, "seen":False}}
N_magic_prev = len(ansatz) #max magic gates
curr_tree = (0,0)
bounds = [(-np.pi/8,np.pi/8)]*len(ansatz)
epsilon = .1
K_path = []

for _ in range(N_K):
    K = K_tree[curr_tree]["K"]
    print("-"*30)
    print(f"K: {K}")
    s = s_dict(N, ansatz, K, order)
    G_K = G_k(N, H, ansatz,K)

    result = scipy.optimize.minimize(energy, theta_init,jac = False, args = (ansatz,s,G_K,order),bounds=bounds)
    magic_indices = np.where(np.abs(result.x)  np.pi/8 < epsilon)
    N_magic = len(magic_indices[0])
    print(f"N_magic: {N_magic}")
    print(f"angles: {result.x}")

    K_tree[curr_tree]["seen"] = True
    K_tree[curr_tree]["N_magic"]=N_magic
    K_path += [K.copy()]

    #if N-magic increases, find new branch skip iteration
    if N_magic > N_magic_prev:
        curr_tree = find_new_branch(K_tree, curr_tree)
        break

    #if all angles within K-cell
    if N_magic == 0:
        break

    #include new branches
    for i in range(N_magic):
        magic_index = magic_indices[0][i]
        sign = np.sign(result.x[magic_index])
        K_i = K_tree[curr_tree]["K"].copy()
        K_i[magic_index] = K_i[magic_index]+int(sign)*1
        K_tree[curr_tree+(i,)]={"K":K_i, "seen":False}
    
    #choose new branch, only fires when N_magic <= N_magic_prev
    assert N_magic <= N_magic_prev, "N_magic > N_magic_prev??"

    updated = False
    for i in range(N_magic):
        magic_index = magic_indices[0][i]
        K_i = K_tree[curr_tree+(i,)]["K"]
    
        if K_i not in K_path: #if not already seen
            curr_tree += (i,) #update tree
            updated = True
            break

    if not updated: #if all new branches have already been visited (loops)
        curr_tree = find_new_branch()
    N_magic_prev = N_magic



    





------------------------------
K: [0, 0, 0]
N_magic: 3
angles: [0.0249792  0.39269908 0.39269908]
------------------------------
K: [1, 0, 0]
N_magic: 3
angles: [-0.39269908  0.39269908  0.39269908]
------------------------------
K: [1, 1, 0]
N_magic: 3
angles: [-3.92699082e-01 -1.20084617e-05 -1.20084617e-05]
------------------------------
K: [0, 1, 0]
N_magic: 3
angles: [ 2.49792023e-02 -6.88387685e-07 -1.39426827e-07]
------------------------------
K: [0, 1, -1]
N_magic: 3
angles: [0.0249792  0.39269908 0.39269908]
------------------------------
K: [1, 1, -1]
N_magic: 3
angles: [-0.39269908  0.39269908  0.39269908]
------------------------------
K: [1, 2, -1]
N_magic: 3
angles: [-3.92699082e-01 -1.20084617e-05 -1.20084617e-05]
------------------------------
K: [0, 2, -1]
N_magic: 3
angles: [ 2.49792023e-02 -6.88387685e-07 -1.39426827e-07]
------------------------------
K: [0, 2, -2]
N_magic: 3
angles: [0.0249792  0.39269908 0.39269908]
------------------------------
K: [1, 2, -2]
N_

In [41]:
import numpy as np
a = np.array([1,2,1,3,5])
print(np.sum(a>1))
print(np.where(a>1))
print(K_tree)
print(K_path)

3
(array([1, 3, 4]),)
{(0, 0): {'K': [0, 0, 0], 'seen': True, 'N_magic': 3}, (0, 0, 0): {'K': [1, 0, 0], 'seen': True, 'N_magic': 3}, (0, 0, 1): {'K': [0, 1, 0], 'seen': False}, (0, 0, 2): {'K': [0, 0, 1], 'seen': False}, (0, 0, 0, 0): {'K': [0, 0, 0], 'seen': False}, (0, 0, 0, 1): {'K': [1, 1, 0], 'seen': True, 'N_magic': 3}, (0, 0, 0, 2): {'K': [1, 0, 1], 'seen': False}, (0, 0, 0, 1, 0): {'K': [0, 1, 0], 'seen': True, 'N_magic': 3}, (0, 0, 0, 1, 1): {'K': [1, 0, 0], 'seen': False}, (0, 0, 0, 1, 2): {'K': [1, 1, -1], 'seen': False}, (0, 0, 0, 1, 0, 0): {'K': [1, 1, 0], 'seen': False}, (0, 0, 0, 1, 0, 1): {'K': [0, 2, 0], 'seen': True, 'N_magic': 3}, (0, 0, 0, 1, 0, 2): {'K': [0, 1, 1], 'seen': False}, (0, 0, 0, 1, 0, 1, 0): {'K': [-1, 2, 0], 'seen': True, 'N_magic': 3}, (0, 0, 0, 1, 0, 1, 1): {'K': [0, 3, 0], 'seen': False}, (0, 0, 0, 1, 0, 1, 2): {'K': [0, 2, 1], 'seen': False}, (0, 0, 0, 1, 0, 1, 0, 0): {'K': [0, 2, 0], 'seen': False}, (0, 0, 0, 1, 0, 1, 0, 1): {'K': [-1, 3, 0], 'se

In [ ]:
#initialize

# N = 1
# t = 6*np.pi/8
# Z_h = -np.cos(t)
# X_h = -np.sin(t)
# ansatz = [pauli("Y0",N)]
# H = [pauli("Z0",N,Z_h),pauli("X0",N,X_h)]



# N = 2
# Z_h = -1
# X_h = -10
# ansatz = [pauli("X0X1",N),pauli("Z0",N),pauli("Z1",N)]
# H = [pauli("Z0",N,Z_h),pauli("Z1",N,Z_h)]
# H+= [pauli("X0X1",N,X_h)]

N = 4
Z_h = -1
X_h = -10
ansatz = [pauli("Y0X1",N),pauli("Y1X2",N),pauli("Y2X3",N)]*3
H = [pauli("Z0",N,Z_h),pauli("Z1",N,Z_h),pauli("Z2",N,Z_h),pauli("Z3",N,Z_h)]
H+= [pauli("X0X1",N,X_h),pauli("X1X2",N,X_h),pauli("X2X3",N,X_h)]



# N = 3
# Z_h = -1
# X_h = -3
# ansatz = [pauli("X0Y1",N),pauli("X1Y2",N)]*2
# H = [pauli("Z0",N,Z_h),pauli("Z1",N,Z_h),pauli("Z2",N,Z_h)]
# H+= [pauli("X0X1",N,X_h),pauli("X1X2",N,X_h)]




H_m = sum([h.matrix_repr() for h in H])
ansatz_m = [a.matrix_repr() for a in ansatz]

iters = 10
K_init = [0]*len(ansatz)
theta_init = (np.random.random(len(ansatz))-.5)*.4
K_path = []
tracking_dictionary = {}
order = 6
# order = 6
epsilon = 2**-5
bounds = [(-3*np.pi/8,3*np.pi/8)]*len(ansatz)
output_method = ""

print(f"Ground-state: {Energy_eigen(H_m)}")
global_min = scipy.optimize.minimize(Energy_matrix, theta_init, jac = False, args = (N,H_m,ansatz_m, K_init))
print(f"Lowest Energy of ansatz: {global_min.fun}, angles: {global_min.x}")
print()
print('-'*20)
print("Start K-cell finding")
print('-'*20)
for i in range(iters):

  print(f"iteration:{i}")
  print(f"K = {K_init}")
  K_path+=[K_init.copy()]
  s = s_dict(N, ansatz, K_init, order)
  G_K = G_k(N, H, ansatz,K_init)
  result = scipy.optimize.minimize(energy, theta_init,jac = False, args = (ansatz,s,G_K,order))
  

  max_index = np.argmax(np.abs(result.x))
  print(f"angles={result.x}, {max_index}")
  print(f"largest angle: {np.abs(result.x[max_index])/np.pi} *pi")
  print(f"Energy intermediate={result.fun}")
  print(f"angle distance: {angle_compare(global_min.x,result.x,K_init)} ")
  print(f"wavefunction distance: {wavefunction_compare(global_min.x,result.x,K_init,ansatz_m)} ")


  angle_norm = np.linalg.norm(result.x)
  tracking_dictionary[f"iteration_{i}"] = {"K": K_init, "E": result.fun, "angles": result.x, "max_index": max_index,"angle_norm":angle_norm}

  if np.abs(result.x[max_index]/np.pi) > 1/8:
    K_init[max_index]+= int(np.sign(result.x[max_index]))
  elif np.abs(np.abs(result.x[max_index]/np.pi) - 1/8) < epsilon:
    break
  else:
    output_method += "ALL ANGLES BELOW PI/8"
    break

  if K_init in K_path:
    print(f"K-cell has already been considered")
    index_repeated_K = K_path.index(K_init)
    print(index_repeated_K)

    #Search for K-cells next biggest angles
    alternative_cell = False
    for j in range(i,index_repeated_K-1,-1):
      angs = tracking_dictionary[f"iteration_{j}"]["angles"]
      second_max_index = np.argsort(angs)[-2]
      if np.abs(result.x[second_max_index])/np.pi > 1/8:
          K_prev = tracking_dictionary[f"iteration_{j}"]["K"]
          K_prev[second_max_index] += int(np.sign(angs[second_max_index]))
          K_init = K_prev.copy()
          alternative_cell = True
          break

    #If there are only 
    if not alternative_cell:
      periodicity = i - index_repeated_K +1
      
      if periodicity == 2:
        previous_angles = tracking_dictionary[f"iteration_{i-1}"]["angles"]
        output_angles = (previous_angles+result.x)/2
        output_method += "MEAN OF 2 PERIODIC K-CELLS"
        break
      else: #periodicity > 2
        norms = [tracking_dictionary[f"iteration_{k}"]["angle_norm"] for k in range(len(tracking_dictionary))]
        mx_index = np.argmax(np.array(norms))
        output_angles = tracking_dictionary[f"iteration_{mx_index}"]["angles"]
        output_method += "K_CELL TAKEN WITH LOWEST ANGLE-NORM"

        
        

  print(f"K changes to --> {K_init}")
  print("-"*20)
  print()
  print()


print("-"*20)
print(f"End of program: angles={result.x}, Energy = {result.fun}, K = {K_init}")
print(f"Reason of termination: {output_method}")
print(f"K_path: {K_path}")










func:'Energy_eigen'  took: 0.0002 sec
Ground-state: (-30.151115237990737, array([ 1.93170048e-14,  1.51172098e-12,  3.34494513e-01, -3.69566817e-01,
       -2.10603613e-14,  1.57541400e-13, -9.07163895e-02,  5.93461215e-01,
        4.47748848e-02,  9.15540745e-14,  2.18512442e-13,  7.15059866e-15,
       -6.09834127e-01, -1.19209766e-01,  5.47237648e-02, -2.91435625e-14]))
Lowest Energy of ansatz: -30.151115237990645, angles: [ 0.82979875  0.06926842 -0.7290178  -0.68650638 -0.76537976 -0.26487964
 -0.74628301 -0.13685027  0.28408146]

--------------------
Start K-cell finding
--------------------
iteration:0
K = [0, 0, 0, 0, 0, 0, 0, 0, 0]
angles=[ 0.60517298 -0.01089184 -0.73746746 -0.63263183 -0.78146196 -0.10734736
 -0.73365283  0.05485046  0.13601223], 4
largest angle: 0.24874706886016215 *pi
Energy intermediate=-30.15111523799029
angle distance: 8.699672046686972 
wavefunction distance: 0.0019705921463541325 
K changes to --> [0, 0, 0, 0, -1, 0, 0, 0, 0]
--------------------


it

In [ ]:
iters = 10
K_init = [0]*len(ansatz)
theta_init = (np.random.random(len(ansatz))-.5)*.4
K_path = []
K_new = [K_init]
tracking_dictionary = {}
order = 6
# order = 6
epsilon = 2**-5
output_method = ""

print(f"Ground-state: {Energy_eigen(H_m)}")
global_min = scipy.optimize.minimize(Energy_matrix, theta_init, jac = False, args = (N,H_m,ansatz_m, K_init))
print(f"Lowest Energy of ansatz: {global_min.fun}, angles: {global_min.x}")
print()
print('-'*20)
print("Start K-cell finding")
print('-'*20)
for i in range(iters):
  for K in K_new:
    K_children = []
    K_path+=[K.copy()]
    s = s_dict(N, ansatz, K, order)
    G_K = G_k(N, H, ansatz,K)
    result = scipy.optimize.minimize(energy, theta_init,jac = False, args = (ansatz,s,G_K,order))

    indices = np.where(result.x > np.pi/8)[0]

    for index in indices:
      K_child = 


  
  K_new = K_children






[0, -1, 0, 0, 1, 0, 0, 0, 1]


In [ ]:
a = np.array([1,2,3,4,5,6])
print(np.where(a >2))

(array([2, 3, 4, 5]),)




*   If cells repeat, choose one, end algorithm
*   What do if theta --> pi/8?


*   How many gates are realistically possible on physical quantum circuit? How many K-cells?
* Where does the randomness come from?
* Why does the energy get lower than the actual ground state energy?



* Compute global minimum of ansatz
* Compute corresponding angles
* Compare angles global minimum to approximated ( multiple minima ?)--> look at psi instead of the angles?

* switch to next biggest angle if periodic behaviour appears




In [ ]:
# a = pauli("Z0",2)
# b = pauli("X0X1",2)

# print(Clifford_map(Clifford_map(Clifford_map(a, b),b),b))
N=2
ansatz = [pauli("X0X1",N),pauli("Z0",N),pauli("Z1",N)]

res = pull_cliffords_through(ansatz, [-2,0,0],N)
for i in res: print(i)

X0X1.   factor: 1
Z0I1.   factor: (-1+0j)
I0Z1.   factor: (-1+0j)


In [ ]:

N = 10
ansatz = [pauli("X0Y1",N),pauli("X1Y2",N),pauli("X2Y3",N),pauli("X3Y4",N),pauli("X4Y5",N),pauli("X5Y6",N),pauli("X6Y7",N),pauli("X7Y8",N),pauli("X8Y9",N)]*2
Z_h = -1
X_h = -1
H = [pauli("Z0",N,Z_h),pauli("Z1",N,Z_h),pauli("Z2",N,Z_h),pauli("Z3",N,Z_h),pauli("Z4",N,Z_h),pauli("Z5",N,Z_h),pauli("Z6",N,Z_h),pauli("Z7",N,Z_h),pauli("Z8",N,Z_h),pauli("Z9",N,Z_h)]
H+=[pauli("X0X1",N,X_h),pauli("X1X2",N,X_h),pauli("X2X3",N,X_h),pauli("X4X5",N),pauli("X5X6",N),pauli("X6X7",N),pauli("X7X8",N),pauli("X8X9",N)]
K = [0]*18


# N = 7
# Z_h = -1
# X_h = -1
# ansatz = [pauli("X0Y1",N),pauli("Z1Z2",N), pauli("X2Y3",N),pauli("X3Y4",N),pauli("X4Y5",N),pauli("Z5Y6",N)]*2
# H = [pauli("Z0",N,Z_h),pauli("Z1",N,Z_h),pauli("X2",N,Z_h),pauli("Y3",N,Z_h),pauli("Z4",N,Z_h),pauli("Z5",N,Z_h),pauli("Y6",N,Z_h)]
# H+= [pauli("X0X1",N,X_h),pauli("X1X2",N,X_h),pauli("X2X3",N,X_h),pauli("X3X4",N,X_h),pauli("X4X5",N,X_h),pauli("X5X6",N,X_h)]
# K = [0]*len(ansatz)


# N = 3
# Z_h = -1
# X_h = -.01
# ansatz = [pauli("Z0Y1",N),pauli("X1Y2",N),pauli("Z0Z1",N)]
# H = [pauli("Z0",N,Z_h),pauli("Z1",N,Z_h),pauli("X2",N,Z_h)]
# H+= [pauli("X0X1",N,X_h),pauli("X1X2",N,X_h)]
# K = [0,1,1]





# N = 2
# Z_h = -1
# X_h = -1
# ansatz = [pauli("X0Y1",N),pauli("Z0I1",N)]
# H = [pauli("Z0",N,Z_h),pauli("Z1",N,Z_h)]
# H= [pauli("X0X1",N,X_h)]
# K = [0,1]



# N = 1
# Z_h = -1
# X_h = -1
# ansatz = [pauli("X0",N),pauli("Y0",N)]
# H = [pauli("Y0",N,Z_h)]
# K = [1,0]


matrix_ansatz = [t.matrix_repr() for t in ansatz]
matrix_H = sum([h.matrix_repr() for h in H])



NameError: ignored

In [ ]:

thetas = np.random.random(len(ansatz))*np.pi/4
# thetas = [np.pi/4,np.pi/4]
# thetas = np.ones(len(ansatz))*np.pi/6
E_full = Energy_matrix(thetas, N, matrix_H, matrix_ansatz, K)
G_K = G_k(N, H, ansatz, K)

E_order = []
o_max=len(ansatz)*2
s = s_dict(N, ansatz, K, o_max)
print(s_dict)
for o in range(1,o_max+1):
    E_approx = energy(thetas, ansatz, s, G_K, o)
    E_order += [E_approx]
    print(f"{o}/{o_max}")
print(E_order)
plt.plot(range(1,o_max+1), np.log(np.abs(E_order-E_full)))
plt.title(f"N = {N}, len(ansatz) = {len(ansatz)}")
plt.ylabel("log(E_order-E_full)")
plt.xlabel("order (o)")